## Testing
The following document carries out a leave one out cross validation.

In [1]:
import pandas as pd
import os
from src.FaceRecogniser import recognise_lbp_hog
from src.FaceRecogniser.create_training_set import create_training_set
from sklearn.metrics import accuracy_score
from IPython.display import Image
import matplotlib.pyplot as plt
from src import Config
import cv2

In [2]:
labels_df = pd.read_csv(os.path.join('dat', 'labels.csv'))

#### Reappearing labels
* Persons can only be recognised in the test set, if they are seen at least once in the training dataset.  
    * Thus, the total dataset must have these people on two different photos.  
* Reappearing labels are candidates for the test data point during leave one out CV

In [3]:
def get_reappearing_labels(labels_df):
    return [label for label in labels_df.label.unique() if labels_df.label.value_counts()[label] >= 2]

In [4]:
def predict_label(filename, labels_df):
    training_set_df = labels_df[labels_df.filename != filename]
    training_filenames = training_set_df.filename.tolist()
    training_labels = training_set_df.label.tolist()
    create_training_set(training_filenames, training_labels)
    
    id, conf = recognise_lbp_hog.predict(filename)
    return id, conf

Doing Leave-One-Out Cross Validation

In [5]:
reappearing_labels = get_reappearing_labels(labels_df)
predictions = []
ground_truth = []
for i, row in enumerate(labels_df.values):
    if row[1] in reappearing_labels:
        # Do Leave one out Cross Validation
        id, conf = predict_label(row[0], labels_df)
        predictions.append(id)
        ground_truth.append(row[1])
#         print(f'Ground truth label: {row[1]}')
#         print(f'Predicted label: {id}\n Prediction Confidence: {conf}\n')

error: OpenCV(4.2.0) /io/opencv_contrib/modules/face/src/eigen_faces.cpp:72: error: (-210:Unsupported format or combination of formats) In the Eigenfaces method all input samples (training images) must be of equal size! Expected 5476 pixels, but was 33489 pixels. in function 'train'


In [ ]:
accuracy_score(predictions, ground_truth)

This accuracy sucks.

## Classification Mistakes
* Clearly the images have a bad resolution
    * I think this is the major bottleneck
* Some people are not directly facing the camera
* Few images are not faces

In [ ]:
plt.rcParams['xtick.labelsize'] = 0
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['ytick.labelsize'] = 0

In [ ]:
cols = 5
rows = len([1 for i in range(len(predictions)) if predictions[i] != ground_truth[i]])//cols+1

fig, ax = plt.subplots(rows, cols, figsize = (20,20))
path = Config.EXTRACTED_FACES_PATH
img_idx = 0

predictions_idx = 0
for i, row in enumerate(labels_df.values):
    if row[1] in reappearing_labels:
        if predictions[predictions_idx] != ground_truth[predictions_idx]:
            img = cv2.imread(os.path.join(path, row[0]))
            ax[img_idx%rows][img_idx//rows].imshow(img)
            img_idx += 1
        predictions_idx += 1

## Classification successes
* Duplication of pictures led to a higher accuracy
    * 7/25 were not duplicates
* The silver lining is that face recognition works great when it's the same image
    * Hence it does work, just not that well
* Not sure how to automatically remove duplicates...
    * Could just do that by hand
    * But any manual process in the pipeline will be hard to automate later
    * Let's just say for now, that the lbg_hog algorithm wasnt very successful

In [ ]:
cols = 5
rows = len([1 for i in range(len(predictions)) if predictions[i] == ground_truth[i]])//cols+1

fig, ax = plt.subplots(rows, cols, figsize = (20,20))
path = Config.EXTRACTED_FACES_PATH
img_idx = 0

predictions_idx = 0
for i, row in enumerate(labels_df.values):
    if row[1] in reappearing_labels:
        if predictions[predictions_idx] == ground_truth[predictions_idx]:
            img = cv2.imread(os.path.join(path, row[0]))
            ax[img_idx%rows][img_idx//rows].imshow(img)
            img_idx += 1
        predictions_idx += 1